In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import os
import sys
import time
import cv2
from PIL import Image
from keras.preprocessing.image import *
from keras.utils.np_utils import to_categorical
from keras.models import load_model
import keras.backend as K

from models import *
from inference import inference


def calculate_iou(model_name, nb_classes, res_dir, label_dir, image_list):
    conf_m = zeros((nb_classes, nb_classes), dtype=float)
    total = 0
    # mean_acc = 0.
    for img_num in image_list:
        img_num = img_num.strip('\n')
        total += 1
        print('#%d: %s' % (total, img_num))
        pred = img_to_array(Image.open('%s/%s.png' % (res_dir, img_num))).astype(int)
        label = img_to_array(Image.open('%s/%s.png' % (label_dir, img_num))).astype(int)
        flat_pred = np.ravel(pred)
        flat_label = np.ravel(label)
        # acc = 0.
        for p, l in zip(flat_pred, flat_label):
            if l == 255:
                continue
            if l < nb_classes and p < nb_classes:
                conf_m[l, p] += 1
            else:
                print('Invalid entry encountered, skipping! Label: ', l,
                      ' Prediction: ', p, ' Img_num: ', img_num)

        #    if l==p:
        #        acc+=1
        #acc /= flat_pred.shape[0]
        #mean_acc += acc
    #mean_acc /= total
    #print 'mean acc: %f'%mean_acc
    I = np.diag(conf_m)
    U = np.sum(conf_m, axis=0) + np.sum(conf_m, axis=1) - I
    IOU = I/U
    meanIOU = np.mean(IOU)
    return conf_m, IOU, meanIOU


def evaluate(model_name, weight_file, image_size, nb_classes, batch_size, val_file_path, data_dir, label_dir,
          label_suffix='.png',
          data_suffix='.jpg'):
    current_dir = os.path.dirname(os.path.realpath('/home/niu/src/Keras-FCN/evaluate.py'))
    save_dir = os.path.join(current_dir, 'Models/'+model_name+'/res/')
    if os.path.exists(save_dir) == False:
        os.mkdir(save_dir)
    fp = open(val_file_path)
    image_list = fp.readlines()
    fp.close()

    start_time = time.time()
    inference(model_name, weight_file, image_size, image_list, data_dir, label_dir, nb_classes=nb_classes, return_results=False, save_dir=save_dir,
              label_suffix=label_suffix, data_suffix=data_suffix)
    duration = time.time() - start_time
    print('{}s used to make predictions.\n'.format(duration))

    start_time = time.time()
    conf_m, IOU, meanIOU = calculate_iou(model_name, nb_classes, save_dir, label_dir, image_list)
    print('IOU: ')
    print(IOU)
    print('meanIOU: %f' % meanIOU)
    print('pixel acc: %f' % (np.sum(np.diag(conf_m))/np.sum(conf_m)))
    duration = time.time() - start_time
    print('{}s used to calculate IOU.\n'.format(duration))

if __name__ == '__main__':
    # model_name = 'Atrous_DenseNet'
    model_name = 'AtrousFCN_Resnet50_16s'
    # model_name = 'DenseNet_FCN'
    weight_file = 'checkpoint_weights.hdf5'
    # weight_file = 'model.hdf5'
    image_size = (512, 512)
    nb_classes = 10
    batch_size = 1
    # dataset = 'VOC2012_BERKELEY'
    dataset = 'IIT-AFF'
    if dataset == 'VOC2012_BERKELEY':
        # pascal voc + berkeley semantic contours annotations
        train_file_path = os.path.expanduser('~/.keras/datasets/VOC2012/combined_imageset_train.txt') #Data/VOClarge/VOC2012/ImageSets/Segmentation
        # train_file_path = os.path.expanduser('~/.keras/datasets/oneimage/train.txt') #Data/VOClarge/VOC2012/ImageSets/Segmentation
        val_file_path   = os.path.expanduser('~/.keras/datasets/VOC2012/combined_imageset_val.txt')
        data_dir        = os.path.expanduser('~/.keras/datasets/VOC2012/VOCdevkit/VOC2012/JPEGImages')
        label_dir       = os.path.expanduser('~/.keras/datasets/VOC2012/combined_annotations')
        label_suffix = '.png'
    if dataset == 'COCO':
        train_file_path = os.path.expanduser('~/.keras/datasets/VOC2012/VOCdevkit/VOC2012/ImageSets/Segmentation/train.txt') #Data/VOClarge/VOC2012/ImageSets/Segmentation
        # train_file_path = os.path.expanduser('~/.keras/datasets/oneimage/train.txt') #Data/VOClarge/VOC2012/ImageSets/Segmentation
        val_file_path   = os.path.expanduser('~/.keras/datasets/VOC2012/VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt')
        data_dir        = os.path.expanduser('~/.keras/datasets/VOC2012/VOCdevkit/VOC2012/JPEGImages')
        label_dir       = os.path.expanduser('~/.keras/datasets/VOC2012/VOCdevkit/VOC2012/SegmentationClass')
        label_suffix = '.npy'
    if dataset == 'IIT-AFF':
        path_prefix = '/home/niu/Liang_Niu3/IIT_Affordances_2017/'
        train_file_path = os.path.join(path_prefix, 'fcn_train_and_val.txt')
        val_file_path   = os.path.join(path_prefix, 'fcn_test.txt')
        # data_dir        = os.path.join(path_prefix, 'rgb_origin')
        data_dir        = os.path.join(path_prefix, 'rgb') # 512x512 images
        target_size = (512, 512)
        label_dir       = os.path.join(path_prefix, 'affordances_labels_png')
        data_suffix='.jpg'
        label_suffix='.png'
        # 0-background,1-contain,2-cut,3-display,4-engine,5-grasp
        # 6-hit,7-pound,8-support,9-w-grasp (wrap grasp)
        classes = 10
        class_weight = None
    print("begin to evaluate")
    evaluate(model_name, weight_file, image_size, nb_classes, batch_size, val_file_path, data_dir, label_dir,
    label_suffix=label_suffix, data_suffix=data_suffix)


/home/niu/anaconda3/envs/keras_fcn/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


begin to evaluate
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (1, 512, 512, 3)     0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (1, 256, 256, 64)    9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (1, 256, 256, 64)    256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (1, 256, 256, 64)    0           bn_conv1[0][0]                   
___________________________________________________________________________________________

#2: ILSVRC2014_train_00012549
#3: ILSVRC2014_train_00019502
#4: n03481172_4671
#5: n03481172_8
#6: n07930864_31337
#7: n03797390_2570
#8: n04039381_15095
#9: n04152593_73469
#10: 18_00000291
#11: img38_00000028
#12: 16_00000459
#13: n04270147_3274
#14: 25_00000249
#15: n04263257_7284
#16: img27_00000042
#17: ILSVRC2014_train_00006472
#18: n03797390_2342
#19: n04591713_5500
#20: n03782006_15910
#21: img37_00000055
#22: n03782006_5382
#23: n03995372_2274
#24: n04591713_4613
#25: n03624134_20217
#26: n03063599_3914
#27: ILSVRC2014_train_00035559
#28: n03481172_5720
#29: n04270147_7548
#30: ILSVRC2014_train_00005588
#31: n04591713_4054
#32: 20_00000477
#33: img1_00000110
#34: n04152593_29607
#35: ILSVRC2014_train_00006601
#36: 08_00000157
#37: n04591713_127
#38: 11_00000141
#39: n03624134_12827
#40: 05_00000263
#41: n04591713_2753
#42: n04270147_18307
#43: n04152593_6855
#44: n03400231_4461
#45: n03782006_12506
#46: ILSVRC2014_train_00033567
#47: 34_00000174
#48: ILSVRC2014_train_00036716


#370: n03782006_8440
#371: ILSVRC2014_train_00002835
#372: 16_00000505
#373: 24_00000108
#374: n03782006_17627
#375: n04152593_24510
#376: ILSVRC2014_train_00040823
#377: n03624134_14121
#378: n04152593_6513
#379: n07930864_35263
#380: n03400231_14768
#381: ILSVRC2014_train_00014796
#382: n03063599_3660
#383: img41_00000100
#384: ILSVRC2014_train_00036610
#385: ILSVRC2014_train_00041077
#386: n03797390_3351
#387: n03775546_3104
#388: ILSVRC2014_train_00051776
#389: ILSVRC2014_train_00053982
#390: 13_00000748
#391: n04591713_5170
#392: 18_00000350
#393: n03995372_5459
#394: n03995372_1919
#395: n03624134_11001
#396: n07930864_2316
#397: 11_00000126
#398: 28_00000202
#399: n03481172_12416
#400: n04270147_1843
#401: n03624134_10136
#402: img27_00000058
#403: img12_00000080
#404: n03400231_20337
#405: n07930864_22504
#406: n03481172_11835
#407: 04_00000500
#408: img7_00000070
#409: n03400231_19597
#410: 31_00000160
#411: ILSVRC2014_train_00047053
#412: n04270147_13739
#413: 01_00000255
#41

#721: n03624134_10316
#722: n03400231_22318
#723: n04039381_11758
#724: n03481172_27528
#725: 04_00000633
#726: n04591713_2666
#727: ILSVRC2014_train_00056219
#728: ILSVRC2014_train_00045722
#729: n03782006_15639
#730: img6_00000048
#731: n04270147_7696
#732: n04152593_61607
#733: n03481172_12093
#734: ILSVRC2014_train_00038873
#735: 28_00000193
#736: n03995372_414
#737: 16_00000353
#738: ILSVRC2014_train_00027021
#739: img20_00000094
#740: img23_00000072
#741: n03624134_18826
#742: img29_00000038
#743: n03400231_6300
#744: n04039381_13402
#745: n04263257_5869
#746: ILSVRC2014_train_00011793
#747: 21_00000472
#748: ILSVRC2014_train_00043393
#749: img1_00000058
#750: n04263257_11
#751: 11_00000335
#752: 12_00000247
#753: ILSVRC2014_train_00046341
#754: n04591713_7385
#755: n03782006_15261
#756: img8_00000111
#757: ILSVRC2014_train_00028414
#758: ILSVRC2014_train_00002305
#759: n07930864_29990
#760: ILSVRC2014_train_00019374
#761: n03782006_6757
#762: 00_00000127
#763: n03400231_1615
#76

...

#1208: n04270147_7910
#1209: n03995372_1987
#1210: n04270147_18034
#1211: ILSVRC2014_train_00042509
#1212: n04591713_7680
#1213: n04152593_6144
#1214: n04039381_22905
#1215: n02880940_20085
#1216: n03624134_18835
#1217: n03481172_3588
#1218: n03797390_3981
#1219: n03782006_6666
#1220: ILSVRC2014_train_00045244
#1221: n03782006_38710
#1222: n04270147_4683
#1223: n04152593_7776
#1224: n03624134_8781
#1225: n04039381_7583
#1226: n04263257_5637
#1227: 35_00000243
#1228: ILSVRC2014_train_00024677
#1229: ILSVRC2014_train_00051144
#1230: ILSVRC2014_train_00049481
#1231: ILSVRC2014_train_00013288
#1232: n03400231_5945
#1233: n04039381_13565
#1234: n03481172_26884
#1235: n03782006_10839
#1236: img11_00000147
#1237: img21_00000065
#1238: ILSVRC2014_train_00015331
#1239: n04152593_24045
#1240: ILSVRC2014_train_00006112
#1241: n04591713_1860
#1242: n04152593_5188
#1243: n04591713_2178
#1244: n04270147_12553
#1245: ILSVRC2014_train_00032830
#1246: n03775546_10103
#1247: n03624134_16935
#1248: img11

#1549: img2_00000067
#1550: n03400231_4937
#1551: n04591713_3570
#1552: n04263257_2644
#1553: n03400231_13824
#1554: 19_00000224
#1555: n03995372_1612
#1556: img4_00000077
#1557: n03400231_1623
#1558: ILSVRC2014_train_00002617
#1559: ILSVRC2014_train_00056504
#1560: n03782006_4514
#1561: ILSVRC2014_train_00046616
#1562: n03775546_5867
#1563: ILSVRC2014_train_00041696
#1564: n04152593_54833
#1565: n04270147_4634
#1566: ILSVRC2014_train_00008810
#1567: n04039381_9885
#1568: n03797390_4206
#1569: img13_00000092
#1570: n07930864_30479
#1571: n03995372_8312
#1572: ILSVRC2014_train_00029426
#1573: n03797390_4150
#1574: img1_00000144
#1575: img22_00000078
#1576: n03400231_3328
#1577: n04039381_8494
#1578: n03481172_32471
#1579: 05_00000271
#1580: ILSVRC2014_train_00001664
#1581: n03400231_14342
#1582: img14_00000094
#1583: ILSVRC2014_train_00049519
#1584: n04152593_36374
#1585: 20_00000325
#1586: n03995372_3932
#1587: n04039381_10043
#1588: n03400231_3806
#1589: 21_00000175
#1590: ILSVRC2014_

#1890: n04270147_7184
#1891: n04591713_1548
#1892: n04263257_7049
#1893: n04152593_19779
#1894: n03481172_28356
#1895: img30_00000088
#1896: n07930864_16155
#1897: 24_00000154
#1898: n02880940_17276
#1899: n03995372_1092
#1900: n04039381_10600
#1901: n03400231_16895
#1902: 14_00000415
#1903: ILSVRC2014_train_00055061
#1904: n03400231_8969
#1905: n03481172_29919
#1906: n02880940_16597
#1907: n03400231_14325
#1908: n04591713_3842
#1909: ILSVRC2014_train_00003735
#1910: 02_00000110
#1911: n03782006_1694
#1912: ILSVRC2014_train_00014398
#1913: ILSVRC2014_train_00045409
#1914: n04270147_4295
#1915: n04591713_19185
#1916: n04270147_2568
#1917: ILSVRC2014_train_00020564
#1918: 38_00000086
#1919: 40_00000195
#1920: ILSVRC2014_train_00054646
#1921: n03481172_39042
#1922: ILSVRC2014_train_00030894
#1923: n04591713_2694
#1924: n04039381_4305
#1925: n03063599_3876
#1926: n04152593_6098
#1927: n03995372_2141
#1928: n07930864_22164
#1929: n04152593_22588
#1930: ILSVRC2014_train_00010191
#1931: n0340

#2229: 32_00000140
#2230: n03481172_11270
#2231: n04039381_652
#2232: 11_00000112
#2233: ILSVRC2014_train_00057183
#2234: n03400231_16130
#2235: n03782006_8044
#2236: ILSVRC2014_train_00031316
#2237: ILSVRC2014_train_00002540
#2238: n03063599_3887
#2239: ILSVRC2014_train_00016790
#2240: n07930864_15697
#2241: n04039381_7115
#2242: n03995372_2650
#2243: n03782006_12158
#2244: 21_00000413
#2245: 31_00000191
#2246: ILSVRC2014_train_00037102
#2247: n03995372_423
#2248: img14_00000047
#2249: n03624134_17511
#2250: n03063599_3916
#2251: n03063599_5248
#2252: n03995372_2327
#2253: ILSVRC2014_train_00038134
#2254: 20_00000218
#2255: n03481172_1999
#2256: ILSVRC2014_train_00055159
#2257: ILSVRC2014_train_00044358
#2258: n03995372_9747
#2259: n04591713_710
#2260: n03400231_19969
#2261: n04152593_37138
#2262: ILSVRC2014_train_00036750
#2263: ILSVRC2014_train_00033321
#2264: ILSVRC2014_train_00040344
#2265: n03775546_3025
#2266: 27_00000515
#2267: ILSVRC2014_train_00015906
#2268: ILSVRC2014_train_

#2572: 31_00000297
#2573: ILSVRC2014_train_00023241
#2574: ILSVRC2014_train_00038472
#2575: img29_00000062
#2576: 13_00000463
#2577: n03782006_12735
#2578: n03400231_13310
#2579: n04152593_7773
#2580: n04270147_12477
#2581: n04263257_8766
#2582: n03782006_11875
#2583: 08_00000125
#2584: 07_00000311
#2585: n03797390_5667
#2586: n04591713_5424
#2587: n07930864_31818
#2588: n04591713_1616
#2589: img22_00000099
#2590: img41_00000060
#2591: n03211117_8879
#2592: n03063599_4194
#2593: n03400231_9152
#2594: ILSVRC2014_train_00008843
#2595: ILSVRC2014_train_00011400
#2596: n04039381_13513
#2597: ILSVRC2014_train_00042512
#2598: img40_00000063
#2599: ILSVRC2014_train_00014404
#2600: n03063599_3751
#2601: n04263257_3161
#2602: 13_00000682
#2603: ILSVRC2014_train_00011738
#2604: img36_00000049
#2605: n04263257_8555
#2606: img11_00000092
#2607: n03624134_10568
#2608: n04039381_15995
#2609: 25_00000163
#2610: 19_00000538
#2611: 17_00000240
#2612: ILSVRC2014_train_00027464
#2613: n03481172_18550
#26